# People Reached

Se realizara el calculo del people reached mediante una seleccion de indicadores, los mismos que sirvieron para el cálculo del submission

In [1]:
import activityinfo as AT
import pandas as pd
import numpy as np
import geopandas as gpd
import time

In [2]:
def get_columns_by_population_type(pop_type, dataframe):
    columns_array=[]
    for col in dataframe.columns:
        if col.startswith(pop_type):
           columns_array.append(col)
    return columns_array

def get_columns_by_age_gender(age_gender, dataframe):
    columns_array=[]
    for col in dataframe.columns:
        if col.endswith(age_gender):
           columns_array.append(col)
    return columns_array


In [3]:
## definimos variables para el calculo mensual
mes_curso = 'AI2022-04.xlsx'
mes = '2022-04'

# Tablas de referencia

In [4]:
MarcoMonitoreo_ref= pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='marco_monitoreo')
indicadores_ref=MarcoMonitoreo_ref[['codigo', 'ppl_reached']]
Plantilla = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\Monitoreo_5w\Templates y guías\Consolidated_Template_2022_Countries.xlsx', sheet_name='Ecuador')
sectores_ref= pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\MasterList\NEW_MasterList2022.xlsx', sheet_name='sectores')

# Leer datos taggeados

In [5]:
df_activity_info = pd.read_excel(r'C:\Users\jorcabrera\OneDrive - International Organization for Migration - IOM\InteragencyGTRM\GTRM 2022\IM 2022\ActivityInfo\DescargasMensuales\procesados\proc_'+mes_curso, index_col=0)


In [6]:
## Join de tablas
df_activity_info = pd.merge(df_activity_info, indicadores_ref, how='left', left_on='codigo',
                                    right_on='codigo')
df_activity_info_CVA = df_activity_info[(df_activity_info['CVA_tag']=='Si')]

# Filtrado de datos
Se filtraran datos que hacen referencia a poblacion CCAA y RRMM, con sus desagregaciones, ademas solamente se consideraran las intervenciones nuevas.

In [7]:
df_activity_info = df_activity_info[df_activity_info['mes']==mes]
df_activity_info = df_activity_info[df_activity_info['ppl_reached']=='Si']
df_activity_info = df_activity_info[df_activity_info['¿esta_actividad_ha_sido_planificada_en_el_rmrp_2022?']=='Si']
df_activity_info = df_activity_info[df_activity_info['frecuencia']=='Nuevo']
df_activity_info = df_activity_info[(df_activity_info['población meta']=='Refugiados y migrantes')|(df_activity_info['población meta']=='Comunidad de acogida')]
df_activity_info = df_activity_info[(df_activity_info['unidad'] == 'Niños') | (df_activity_info['unidad'] == 'Niñas') | (df_activity_info['unidad'] == 'Hombres')| (df_activity_info['unidad'] == 'Mujeres')| (df_activity_info['unidad'] == 'Otros')]
df_activity_info['aux_ppl'] = df_activity_info['población meta'] + '-' + df_activity_info['unidad']


Filtrado de datos CVA

In [8]:
df_activity_info_CVA = df_activity_info_CVA[df_activity_info_CVA['mes']==mes]
df_activity_info_CVA = df_activity_info_CVA[df_activity_info_CVA['ppl_reached']=='Si']
df_activity_info = df_activity_info[df_activity_info['¿esta_actividad_ha_sido_planificada_en_el_rmrp_2022?']=='Si']
df_activity_info_CVA = df_activity_info_CVA[df_activity_info_CVA['frecuencia']=='Nuevo']
df_activity_info_CVA = df_activity_info_CVA[(df_activity_info_CVA['población meta']=='Refugiados y migrantes')|(df_activity_info_CVA['población meta']=='Comunidad de acogida')]
df_activity_info_CVA = df_activity_info_CVA[(df_activity_info_CVA['unidad'] == 'Niños') | (df_activity_info_CVA['unidad'] == 'Niñas') | (df_activity_info_CVA['unidad'] == 'Hombres')| (df_activity_info_CVA['unidad'] == 'Mujeres')| (df_activity_info_CVA['unidad'] == 'Otros')]
df_activity_info_CVA['aux_ppl'] = df_activity_info_CVA['población meta'] + '-' + df_activity_info_CVA['unidad']

# Reumen de datos para people reached

Datos globales

In [9]:
intersector_sum_indicador=df_activity_info.groupby(['mes','sector','DPA_DESPRO','DPA_DESCAN','Acronimo Implementador','población meta', 'unidad', 'indicador'])['Value'].agg('sum','count').reset_index()
intersector_sum_org=intersector_sum_indicador.groupby(['mes','sector','DPA_DESPRO','DPA_DESCAN','Acronimo Implementador','población meta', 'unidad'])['Value'].agg(['max', 'count']).reset_index()
intersector_max_canton=intersector_sum_org.groupby(['mes','sector','DPA_DESPRO','DPA_DESCAN','población meta', 'unidad'])['max'].agg(['max']).reset_index()

In [10]:
intersector_max_canton['aux'] = intersector_max_canton['población meta'] + '-' + intersector_max_canton['unidad']
sector_mes = intersector_max_canton.groupby(['mes','sector','DPA_DESPRO','aux'])['max'].agg('sum').unstack(fill_value=0).reset_index().rename_axis(None, axis=1)
sector_mes['DPA_DESPRO'] = sector_mes['DPA_DESPRO'].str.title()
cols_intersector=sector_mes.columns[3:]
INTERSECTOR = sector_mes
sector_mes = pd.merge(sector_mes, sectores_ref, how='left', left_on=['sector'],
                                    right_on=['sector'])
cols = get_columns_by_population_type('Refugiados y migrantes', sector_mes)
sector_mes['Consolidated In Destination'] = sector_mes[cols].sum(axis=1)
cols = get_columns_by_population_type('Comunidad de acogida', sector_mes)
sector_mes['Consolidated Host Communities'] = sector_mes[cols].sum(axis=1)
cols = get_columns_by_age_gender('Niñas', sector_mes)
sector_mes['Consolidated Girls'] = sector_mes[cols].sum(axis=1)
cols = get_columns_by_age_gender('Niños', sector_mes)
sector_mes['Consolidated Boys'] = sector_mes[cols].sum(axis=1)
cols = get_columns_by_age_gender('Mujeres', sector_mes)
sector_mes['Consolidated Women'] = sector_mes[cols].sum(axis=1)
cols = get_columns_by_age_gender('Hombres', sector_mes)
sector_mes['Consolidated Men'] = sector_mes[cols].sum(axis=1)
cols = get_columns_by_age_gender('Otros', sector_mes)
sector_mes['Consolidated Other above 18'] = sector_mes[cols].sum(axis=1)


INTERSECTOR = INTERSECTOR.groupby(['mes','DPA_DESPRO'])[cols_intersector].agg('max').reset_index().rename_axis(None, axis=1)

cols = get_columns_by_population_type('Refugiados y migrantes', INTERSECTOR)
INTERSECTOR['Consolidated In Destination'] = INTERSECTOR[cols].sum(axis=1)
cols = get_columns_by_population_type('Comunidad de acogida', INTERSECTOR)
INTERSECTOR['Consolidated Host Communities'] = INTERSECTOR[cols].sum(axis=1)
cols = get_columns_by_age_gender('Niñas', INTERSECTOR)
INTERSECTOR['Consolidated Girls'] = INTERSECTOR[cols].sum(axis=1)
cols = get_columns_by_age_gender('Niños', INTERSECTOR)
INTERSECTOR['Consolidated Boys'] = INTERSECTOR[cols].sum(axis=1)
cols = get_columns_by_age_gender('Mujeres', INTERSECTOR)
INTERSECTOR['Consolidated Women'] = INTERSECTOR[cols].sum(axis=1)
cols = get_columns_by_age_gender('Hombres', INTERSECTOR)
INTERSECTOR['Consolidated Men'] = INTERSECTOR[cols].sum(axis=1)
cols = get_columns_by_age_gender('Otros', INTERSECTOR)
INTERSECTOR['Consolidated Other above 18'] = INTERSECTOR[cols].sum(axis=1)
INTERSECTOR.drop(['Comunidad de acogida-Hombres',
'Comunidad de acogida-Mujeres',
'Comunidad de acogida-Niñas',
'Comunidad de acogida-Niños',
'Comunidad de acogida-Otros',
'Refugiados y migrantes-Hombres',
'Refugiados y migrantes-Mujeres',
'Refugiados y migrantes-Niñas',
'Refugiados y migrantes-Niños',
'Refugiados y migrantes-Otros'], axis=1, inplace=True)
INTERSECTOR['Subsector']='Intersector'
INTERSECTOR['Platform']='Ecuador'
INTERSECTOR['Country']='Ecuador'
INTERSECTOR.rename(columns = {'DPA_DESPRO':'Admin1'}, inplace = True)
INTERSECTOR = INTERSECTOR[['Platform',
'Country',
'Admin1',
'Subsector',
'mes',
'Consolidated In Destination',
'Consolidated Host Communities',
'Consolidated Girls',
'Consolidated Boys',
'Consolidated Women',
'Consolidated Men',
'Consolidated Other above 18']]

Datos CVA

In [11]:
intersector_sum_indicador_CVA=df_activity_info_CVA.groupby(['mes','sector','DPA_DESPRO','DPA_DESCAN','Acronimo Implementador','población meta', 'unidad', 'indicador'])['Value'].agg('sum','count').reset_index()
intersector_sum_org_CVA=intersector_sum_indicador_CVA.groupby(['mes','sector','DPA_DESPRO','DPA_DESCAN','Acronimo Implementador','población meta', 'unidad'])['Value'].agg(['max', 'count']).reset_index()
intersector_max_canton_CVA=intersector_sum_org_CVA.groupby(['mes','sector','DPA_DESPRO','DPA_DESCAN','población meta', 'unidad'])['max'].agg(['max']).reset_index()

In [12]:
#CVA_sector_mes = intersector_max_canton_CVA.pivot_table(index=['mes','sector','DPA_DESPRO'],columns=['población meta', 'unidad'], values='max', aggfunc='sum', fill_value=0, margins=True, margins_name='Monthly CVA').reset_index()
intersector_max_canton_CVA['aux_cva'] = intersector_max_canton_CVA['población meta'] + '-' + intersector_max_canton_CVA['unidad']
CVA_sector_mes = intersector_max_canton_CVA.groupby(['mes','sector','DPA_DESPRO','aux_cva'])['max'].agg('sum').unstack(fill_value=0).reset_index().rename_axis(None, axis=1)
INTERSECTOR_CVA = CVA_sector_mes
CVA_sector_mes['Monthly CVA'] = CVA_sector_mes.sum(axis=1)
CVA_sector_mes = CVA_sector_mes[['mes', 'sector', 'DPA_DESPRO', 'Monthly CVA']]
CVA_sector_mes['DPA_DESPRO'] = CVA_sector_mes['DPA_DESPRO'].str.title()

CVA_sector_mes = pd.merge(CVA_sector_mes, sectores_ref, how='left', left_on=['sector'],
                                    right_on=['sector'])


<ipython-input-12-0a0b0b369500>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  CVA_sector_mes['DPA_DESPRO'] = CVA_sector_mes['DPA_DESPRO'].str.title()


In [13]:
## INTERSECTOR CVA

cols_intersector=CVA_sector_mes.columns[3:]
INTERSECTOR_CVA = INTERSECTOR_CVA.groupby(['mes','DPA_DESPRO'])['Monthly CVA'].agg('max').reset_index().rename_axis(None, axis=1)
INTERSECTOR_CVA['DPA_DESPRO'] = INTERSECTOR_CVA['DPA_DESPRO'].str.title()
INTERSECTOR_CVA['Subsector']='Intersector'
INTERSECTOR_CVA['Platform']='Ecuador'
INTERSECTOR_CVA['Country']='Ecuador'
INTERSECTOR_CVA.rename(columns = {'DPA_DESPRO':'Admin1'}, inplace = True)
INTERSECTOR_CVA = INTERSECTOR_CVA[['Admin1',
'Subsector',
'Monthly CVA']]
display(INTERSECTOR_CVA)


,Admin1,Subsector,Monthly CVA
0,Azuay,Intersector,465.0
1,Bolivar,Intersector,15.0
2,Carchi,Intersector,1192.0
3,Cañar,Intersector,8.0
4,Chimborazo,Intersector,61.0
5,Cotopaxi,Intersector,110.0
6,El Oro,Intersector,1680.0
7,Esmeraldas,Intersector,94.0
8,Guayas,Intersector,1233.0
9,Imbabura,Intersector,393.0


# Union de tablas y Join

In [14]:

Plantilla = Plantilla[['Platform', 'Country', 'Admin1', 'Subsector']]
INTERSECTOR_Plantilla = Plantilla[Plantilla['Subsector'] == 'Intersector']
Plantilla = Plantilla[Plantilla['Subsector'] != 'Intersector']

INTERSECTOR_CVA = pd.merge(INTERSECTOR_Plantilla, INTERSECTOR_CVA, how='left', left_on=['Admin1', 'Subsector'],
                                    right_on=['Admin1','Subsector'])
INTERSECTOR_CVA=INTERSECTOR_CVA[['Admin1','Subsector','Monthly CVA']]
INTERSECTOR_CVA = INTERSECTOR_CVA.fillna(0)

## Join de tablas
people_reached_report = pd.merge(Plantilla, CVA_sector_mes, how='left', left_on=['Admin1', 'Subsector'],
                                    right_on=['DPA_DESPRO','Subsector'])

people_reached_report.drop(['mes','sector', 'DPA_DESPRO'], axis=1, inplace=True)

people_reached_report = pd.merge(people_reached_report, sector_mes, how='left', left_on=['Admin1', 'Subsector'],
                                    right_on=['DPA_DESPRO','Subsector'])
people_reached_report.drop(['sector',
'DPA_DESPRO',
'Comunidad de acogida-Hombres',
'Comunidad de acogida-Mujeres',
'Comunidad de acogida-Niñas',
'Comunidad de acogida-Niños',
'Comunidad de acogida-Otros',
'Refugiados y migrantes-Hombres',
'Refugiados y migrantes-Mujeres',
'Refugiados y migrantes-Niñas',
'Refugiados y migrantes-Niños',
'Refugiados y migrantes-Otros'], axis=1, inplace=True)
people_reached_report['mes']= mes
people_reached_report.rename(columns = {'mes':'Month'}, inplace = True)
INTERSECTOR.rename(columns = {'mes':'Month'}, inplace = True)
INTERSECTOR = pd.merge(INTERSECTOR_Plantilla, INTERSECTOR, how='left', left_on=['Admin1', 'Subsector'],
                                    right_on=['Admin1','Subsector'])
#INTERSECTOR=INTERSECTOR[['Admin1','Subsector']]
INTERSECTOR.drop(['Platform_y','Country_y'], axis=1, inplace=True)
INTERSECTOR.rename(columns = {'Platform_x':'Platform','Country_x':'Country' }, inplace = True)
INTERSECTOR_TOTAL = pd.merge(INTERSECTOR, INTERSECTOR_CVA, how='left', left_on=['Admin1', 'Subsector'],
                                    right_on=['Admin1','Subsector'])

people_reached_report = pd.concat([people_reached_report, INTERSECTOR_TOTAL], ignore_index=True)
people_reached_report=people_reached_report[['Platform', 'Country', 'Admin1', 'Month', 'Subsector','Monthly CVA', 'Consolidated In Destination', 'Consolidated Host Communities','Consolidated Girls','Consolidated Boys','Consolidated Women','Consolidated Men','Consolidated Other above 18']]
people_reached_report['Month'] = mes
people_reached_report = people_reached_report.fillna(0)
people_reached_report.to_clipboard(excel=True)


# Intersector
Se toman los maximos entre cada provincia comparando entre los sectores